<a href="https://colab.research.google.com/github/sh20022002/small-Language-Model/blob/main/slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving tokenizer.zip to tokenizer.zip


In [22]:
import zipfile, os, sys

# Unzip the uploaded file
with zipfile.ZipFile("tokenizer.zip", 'r') as zip_ref:
    zip_ref.extractall("tokenizer")

# Add to sys.path so Python can find your modules
sys.path.append(os.path.abspath("tokenizer"))


In [23]:
!pip install datasets

In [25]:
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")


In [28]:
dataset['train']

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [29]:
# @markdown Dependency: 5Jwk6myk_LdW
from tokenizer import HybridTokenizer, Transformer, train_model
import torch

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
!ls tokenizer/tokenizer/


create_t_f.py	       __pycache__  slm.ipynb		    train.py
hybrid_tokeniztion.py  README.md    tokenizer_state.pkl.gz  transformer.py
__init__.py	       scrshots     tok_test.ipynb


In [32]:
tok = HybridTokenizer.load('tokenizer/tokenizer/tokenizer_state.pkl.gz')

In [33]:
dataset = dataset['train']

In [51]:
dataset[500]

{'instruction': 'Describe the character of Romeo from Romeo and Juliet.',
 'input': '',
 'output': 'Romeo is a tragic hero with a passionate heart. He is brave and impulsive, often making decisions without fully thinking them through. He is deeply romantic and has a heartbreaking loyalty to his family and his beloved, Juliet.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe the character of Romeo from Romeo and Juliet.\n\n### Response:\nRomeo is a tragic hero with a passionate heart. He is brave and impulsive, often making decisions without fully thinking them through. He is deeply romantic and has a heartbreaking loyalty to his family and his beloved, Juliet.'}

In [52]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Prepare raw text
        prompt = f"Q: {item['instruction']}\nA:"
        target = item['output']

        # Encode input and target using your tokenizer
        input_ids = self.tokenizer.encode(prompt, mode="flat")[:self.max_length]
        label_ids = self.tokenizer.encode(target, mode="flat")[:self.max_length]

        # Convert to tensors
        input_tensor = torch.tensor(input_ids, dtype=torch.long)
        label_tensor = torch.tensor(label_ids, dtype=torch.long)

        return {
            "input_ids": input_tensor,
            "labels": label_tensor
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = [item["labels"] for item in batch]

    padded_inputs = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0)
    padded_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    # Create attention mask: 1 where token is not padding (value is not 0), 0 otherwise
    attention_mask = (padded_inputs != 0).long()

    return {
        "input_ids": padded_inputs,
        "labels": padded_labels,
        "attention_mask": attention_mask # Add attention mask here
    }



In [53]:
# Split the dataset using the datasets library's built-in method
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Access the splits
train_data_list = split_dataset['train']
val_data_list = split_dataset['test'] # The split method names the validation split 'test' by default

# Create QADataset instances from the split data
train_dataset = QADataset(train_data_list, tok)
val_dataset = QADataset(val_data_list, tok)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

In [ ]:
from torch.optim import AdamW
model = Transformer(
    vocab_size=len(tok.token2id),
    dim=512,
    depth=6,
    heads=8,
    mlp_dim=2048,
    window=256
).to(device)


optimizer = AdamW(model.parameters(), lr=3e-4)

vocab = tok.vocab_size
model.resize_token_embeddings(vocab)
assert model.token_emb.num_embeddings == vocab


train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    epochs=5
)


ValueError: too many values to unpack (expected 3)

The `TypeError` in the `train_test_split` function arises because `sklearn`'s `train_test_split` is not designed to directly handle custom PyTorch `Dataset` objects. To resolve this, we will first split the underlying data (the list of dictionaries) into training and validation sets using `train_test_split`, and then create `QADataset` instances for each split.